# MLofi: Predictions

## Setup

### Hyperparameters

In [1]:
sequence_length = 30
# possible values ["srnn", "lstm", "gru"]
model_type = "lstm"

### Imports

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate, Activation

from tensorflow.keras.layers import SimpleRNN, GRU

from music21 import *

import numpy as np

import os
import re

### Mount Google Drive to access the dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def create_path(relative_path):
  return f'/content/drive/My Drive/MLofi/{relative_path}'

In [5]:
model_weights_path = create_path('weights/weights.hdf5')
path_to_unique_notes = create_path('output/data/unique_notes.txt')

### Read the note, chord and rest collection

In [6]:
with open(path_to_unique_notes, "r") as f:
  unique_notes = f.read().split("\n")
note_to_index = dict([(note, index) for index, note in enumerate(unique_notes)])
unique_note_count = len(unique_notes)

## Defining the model

In [7]:
if model_type == "lstm":
  input = Input(shape=(sequence_length, unique_note_count))
  hidden_1 = Dropout(0.3)(LSTM(700, return_sequences=True, recurrent_dropout=0.2)(input))
  hidden_2 = Dropout(0.3)(LSTM(700, return_sequences=True, recurrent_dropout=0.2)(Concatenate()([input, hidden_1])))
  hidden_3 = Dropout(0.3)(LSTM(700, return_sequences=True, recurrent_dropout=0.2)(Concatenate()([input, hidden_2])))
  distribution_parameters = LSTM(700, recurrent_dropout=0.3)(Concatenate()([hidden_1, hidden_2, hidden_3]))
  distribution = Dense(unique_note_count)(distribution_parameters)
elif model_type == "gru":
  input = Input(shape=(sequence_length, unique_note_count))
  hidden_1 = Dropout(0.3)(GRU(700, return_sequences=True, recurrent_dropout=0.3)(input))
  hidden_2 = Dropout(0.3)(GRU(700, return_sequences=True, recurrent_dropout=0.3)(Concatenate()([input, hidden_1])))
  hidden_3 = Dropout(0.3)(GRU(700, return_sequences=True, recurrent_dropout=0.3)(Concatenate()([input, hidden_2])))
  distribution_parameters = GRU(700, recurrent_dropout=0.3)(Concatenate()([hidden_1, hidden_2, hidden_3]))
  distribution = Dense(unique_note_count)(distribution_parameters)
elif model_type == "srnn":
  input = Input(shape=(sequence_length, unique_note_count))
  hidden_1 = Dropout(0.3)(SimpleRNN(700, return_sequences=True)(input))
  hidden_2 = Dropout(0.3)(SimpleRNN(700, return_sequences=True)(Concatenate()([input, hidden_1])))
  hidden_3 = Dropout(0.3)(SimpleRNN(700, return_sequences=True)(Concatenate()([input, hidden_2])))
  distribution_parameters = SimpleRNN(700)(Concatenate()([hidden_1, hidden_2, hidden_3]))
  distribution = Dense(unique_note_count)(distribution_parameters)

output = Activation('softmax')(distribution)
model = keras.Model(inputs=input, outputs=output, name='MLofi')

In [10]:
model.summary()

Model: "MLofi"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30, 249)]    0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 30, 700)      2660000     ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 30, 700)      0           ['lstm[0][0]']                   
                                                                                                  
 concatenate (Concatenate)      (None, 30, 949)      0           ['input_1[0][0]',                
                                                                  'dropout[0][0]']            

In [11]:
model.load_weights(model_weights_path)

## Predictions

In [12]:
def predict_from_seed(seed, note_count=500):
  pattern = list(seed)
  prediction = []
  for note_index in range(500):
    x = keras.utils.to_categorical(np.reshape(pattern, (1, sequence_length, 1)), num_classes=unique_note_count)
    p = model.predict(x, verbose=0)

    i = np.argmax(p)
    prediction.append(unique_notes[i])

    pattern.append(i)
    pattern = pattern[1:len(pattern)]
  return prediction

In [13]:
chord_pattern = re.compile(r'\.')
rest_pattern = re.compile(r'Rest')
note_pattern = re.compile(r'[A-Z]')

def process_prediction(prediction):
  s = stream.Stream()
  for element in prediction:
    if element == 'Padding':
      continue
    if rest_pattern.search(element) is not None:
      s.append(note.Rest(element))
    elif chord_pattern.search(element) is not None:
      s.append(chord.Chord([int(x) for x in element.split('.')]))
    elif note_pattern.search(element) is not None:
      s.append(note.Note(element))
  return s

### Random pattern

In [15]:
seed = np.random.randint(low=0, high=unique_note_count, size=sequence_length).tolist()
s = process_prediction(predict_from_seed(seed))
s.write('midi', fp=create_path('output/test/05-01-2022-05-LSTM-withRecurentDropout.mid'))

'/content/drive/My Drive/MLofi/output/test/05-01-2022-05-LSTM-withRecurentDropout.mid'